<a href="https://colab.research.google.com/github/alok1409/British-Airways-Virtual-Internship-/blob/main/British_Airways_Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>